In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [4]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2831) (12128, 2831)


In [5]:
features_list=[
'statedIncome',
'deviation_from_median_mean_income#last_365_Days',
'deviation_from_mean_income#last_730_Days',
'deviation_from_median_age/standard_deviation_income#last_365_Days',
'age',
'deviation_from_mean_income#last_30_Days',
'yearOfBirth_mean',

'deviation_from_mean_age#last_365_Days',
'mean_loan_amount#last_365_Days',
 'authorized_capital',
'year_of_registration',
'month_year',
'90_in_24mob']

In [6]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3024691/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3024691/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 11), (15604, 11), (23406,), (15604,))

In [9]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 11) (4682, 11) (15604, 11)


In [10]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)

In [11]:

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,

    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'lambda': 4,
    'alpha': 4.8,
    'gamma': 5.6,

    'scale_pos_weight': 3.2,
    'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 200,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.54755	eval-auc:0.52284
[10]	train-auc:0.56121	eval-auc:0.54640
[20]	train-auc:0.56237	eval-auc:0.54247
[30]	train-auc:0.56517	eval-auc:0.54181
[40]	train-auc:0.56639	eval-auc:0.53897
[50]	train-auc:0.56770	eval-auc:0.53922


In [12]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.57, Gini: 0.14, KS: 0.10
Eval AUC: 0.54, Gini: 0.08, KS: 0.09
Test AUC: 0.56, Gini: 0.12, KS: 0.08
OOT AUC: 0.54, Gini: 0.09, KS: 0.07


In [13]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.241028         0.420902        0.450774        0.412574   1867   
9          0.210863         0.405365        0.412574        0.399309   1878   
8          0.198118         0.396574        0.399300        0.394397   1807   
7          0.169080         0.392906        0.394395        0.391538   1934   
6          0.154584         0.390649        0.391527        0.389743   1876   
5          0.159218         0.388923        0.389738        0.388091   1790   
4          0.151407         0.386892        0.388083        0.385064   1955   
3          0.121467         0.384434        0.385061        0.383773   1309   
2          0.144281         0.382982        0.383768        0.381435   2308   
1          0.128000         0.380509        0.381430        0.376130   2000   

        label_good  label_bad  
Decile                         
10            1417        450  
9             1482        396  
8             1449        358  
7             1607        327  
6             1586        290  
5             1505        285  
4             1659        296  
3             1150        159  
2             1975        333  
1             1744        256

In [14]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.255305         0.420955        0.450774        0.412544   1555   
9          0.185824         0.405357        0.412543        0.399033   1566   
8          0.188204         0.396436        0.399027        0.394397   1509   
7          0.162896         0.393102        0.394395        0.391838   1547   
6          0.178462         0.390888        0.391835        0.389985   1625   
5          0.159947         0.389070        0.389983        0.388203   1513   
4          0.154947         0.387037        0.388198        0.385097   1607   
3          0.141115         0.384439        0.385094        0.383771   1148   
2          0.150862         0.382968        0.383768        0.381435   1856   
1          0.131108         0.380494        0.381430        0.376130   1678   

        label_good  label_bad  
Decile                         
10            1158        397  
9             1275        291  
8             1225        284  
7             1295        252  
6             1335        290  
5             1271        242  
4             1358        249  
3              986        162  
2             1576        280  
1             1458        220

In [15]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.236603         0.422360        0.450774        0.412334   1213   
9          0.202803         0.406587        0.412331        0.401189   1213   
8          0.171476         0.397415        0.401151        0.394858   1213   
7          0.157591         0.393228        0.394851        0.391933   1212   
6          0.160758         0.390898        0.391933        0.389961   1213   
5          0.160133         0.389020        0.389960        0.388201   1199   
4          0.178396         0.387368        0.388198        0.385678   1222   
3          0.162378         0.384622        0.385675        0.383771   1127   
2          0.161390         0.383087        0.383768        0.381772   1295   
1          0.131040         0.380742        0.381767        0.376130   1221   

        label_good  label_bad  
Decile                         
10             926        287  
9              967        246  
8             1005        208  
7             1021        191  
6             1018        195  
5             1007        192  
4             1004        218  
3              944        183  
2             1086        209  
1             1061        160

In [16]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
6                                    yearOfBirth_mean        44.0
0                                        statedIncome        25.0
8                      mean_loan_amount#last_365_Days        19.0
1     deviation_from_median_mean_income#last_365_Days        15.0
2            deviation_from_mean_income#last_730_Days        14.0
7               deviation_from_mean_age#last_365_Days        14.0
4                                                 age        14.0
3   deviation_from_median_age/standard_deviation_i...         7.0
5             deviation_from_mean_income#last_30_Days         2.0
9                                  authorized_capital         1.0
10                               year_of_registration         1.0


In [17]:
feature_importance_df

Feature  Importance
6                                    yearOfBirth_mean        44.0
0                                        statedIncome        25.0
8                      mean_loan_amount#last_365_Days        19.0
1     deviation_from_median_mean_income#last_365_Days        15.0
2            deviation_from_mean_income#last_730_Days        14.0
7               deviation_from_mean_age#last_365_Days        14.0
4                                                 age        14.0
3   deviation_from_median_age/standard_deviation_i...         7.0
5             deviation_from_mean_income#last_30_Days         2.0
9                                  authorized_capital         1.0
10                               year_of_registration         1.0

In [18]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_LA_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_LA_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_LA_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_LA_Oot_Pred_9024.csv',index=False)


In [19]:
#Save the Model
xgb_model.save_model('LeoPayu_lA_90_in_24mob.json')
print('Model Saved')

Model Saved
